In [1]:
# imports
import tensorflow as tf
import numpy as np
import alpaca_trade_api as tradeapi
from sklearn.model_selection import train_test_split
import neptune
import pandas as pd
from fredapi import Fred
import json
from pathlib import Path

In [2]:
# load stock data and correct index so it matches the macro-economic data
df = pd.read_csv('stock_data_full_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)
df['timestamp'] = df['timestamp'].dt.date
df.set_index('timestamp', inplace=True)

In [3]:
# load bins
df_bin1 = pd.read_csv('bin1_stocks.csv')
df_bin2 = pd.read_csv('bin2_stocks.csv')
df_bin3 = pd.read_csv('bin3_stocks.csv')
df_bin4 = pd.read_csv('bin4_stocks.csv')
df_bin5 = pd.read_csv('bin5_stocks.csv')

In [4]:
bins = [df_bin1, df_bin2, df_bin3, df_bin4, df_bin5]

In [5]:
# drop unneccessary column
for bin in bins:
    bin.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
# Prepare (mostly macro-economic) Features that are used by every stock or ETF

# Prepare macro-economic features
credentials = json.loads(Path('api_Keys.json').read_text())
fred_api_key = credentials['fred_api']['api_key']
fred = Fred(api_key=fred_api_key)
start_date = '2020-05-08'
end_date = '2025-05-07'

# list the indicators we want to download
indicators = {
    "real_GDP": "GDPC1",
    "unemployment_rate": "UNRATE",
    "CPI": "CPIAUCSL",
    "personal_consumption": "PCE",
    "industrial_production": "INDPRO",
    "federal_funds_rate": "FEDFUNDS",
    "10Y_treasury_rate": "DGS10",
    "retail_sales": "RSAFS",
    "housing_starts": "HOUST",
    "usd_eur": "DEXUSEU",
    "usd_gbp": "DEXUSUK",
    "usd_jpy": "DEXJPUS"
}

# iterate to get data for each indicator
data = {}
for name, series_id in indicators.items():
    print(f"Downloading {name}...")
    series_data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
    series_data = series_data.to_frame(name=name)
    data[name] = series_data

# Combine all macro-economic indicators into a single DataFrame
df_macro = pd.concat(data.values(), axis=1)
df_macro = df_macro.interpolate(
    method='linear',
    axis=0,
    limit_direction='both'
)

# cut off all previous data to match the stock data
df_macro = df_macro[df_macro.index >= '2020-05-08']
# write out the macro-economic data
df_macro.to_csv("macro_data.csv", index=True)

In [7]:
df_macro = pd.read_csv("macro_data.csv")

In [8]:
# pipeline to compute technical indicators and macro-economic features for each asset
chosen_stocks = []
for bin in bins:
    df_bin = []
    for ticker in bin.columns:
        df_ticker = df[df['symbol'] == ticker].copy()



        # ---- Add technical indicators ---- #

        # Add Relative Strength Index (RSI)
        window = 14
        delta = df_ticker['close'].diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.ewm(alpha=1/window, min_periods=window).mean()
        avg_loss = loss.ewm(alpha=1/window, min_periods=window).mean() 
        rs = avg_gain / avg_loss
        df_ticker['rsi'] = 100 - (100 / (1 + rs))

        # Add Rate of Change (ROC)
        window1 = 9
        window2 = 14
        window3 = 21
        prev_price1 = df_ticker['close'].shift(window1)
        df_ticker['roc_9'] = (df_ticker['close'] - prev_price1) / prev_price1 * 100
        prev_price2 = df_ticker['close'].shift(window2)
        df_ticker['roc_14'] = (df_ticker['close'] - prev_price2) / prev_price2 * 100
        prev_price3 = df_ticker['close'].shift(window3)
        df_ticker['roc_21'] = (df_ticker['close'] - prev_price3) / prev_price3 * 100

        # Add Moving Average Convergence Divergence (MACD)
        short_span = 12
        long_span = 26
        signal_span = 9
        ema_short = df_ticker['close'].ewm(span=short_span, adjust=False).mean()
        ema_long = df_ticker['close'].ewm(span=long_span, adjust=False).mean()
        df_ticker['macd'] = ema_short - ema_long
        df_ticker['macd_signal'] = df_ticker['macd'].ewm(span=signal_span, adjust=False).mean()
        df_ticker['macd_hist'] = df_ticker['macd'] - df_ticker['macd_signal']

        # Add Stochastic Oscillator
        window_K = 14 # for %K
        window_D = 3
        highest_high = df_ticker['high'].rolling(window=window_K).max()
        lowest_low = df_ticker['low'].rolling(window=window_K).min()
        df_ticker['stoch_%K'] = (df_ticker['close'] - lowest_low) / (highest_high - lowest_low) * 100
        df_ticker['stoch_%D'] = df_ticker['stoch_%K'].rolling(window=window_D).mean()
        df_ticker['stoch_diff'] = df_ticker['stoch_%K'] - df_ticker['stoch_%D']

        # Add Commodity Channel Index (CCI)
        window = 20
        typical_price = (df_ticker['high'] + df_ticker['low'] + df_ticker['close']) / 3
        sma = typical_price.rolling(window=window, min_periods=int(window/2)).mean()
        mad = (typical_price - sma).abs().rolling(window=window).mean()
        df_ticker['cci'] = (typical_price - sma) / (0.015 * mad)

        # Add various Moving Averages
        df_ticker['ma_15'] = df_ticker['close'].rolling(window=15, min_periods=5).mean()
        df_ticker['ma_30'] = df_ticker['close'].rolling(window=30, min_periods=10).mean()
        df_ticker['ma_60'] = df_ticker['close'].rolling(window=60, min_periods=15).mean()
        df_ticker['ma_100'] = df_ticker['close'].rolling(window=100, min_periods=20).mean()
        df_ticker['ma_200'] = df_ticker['close'].rolling(window=200, min_periods=30).mean()

        # Add Average True Range (ATR)
        window = 14
        high_low = df_ticker['high'] - df_ticker['low']
        high_close = (df_ticker['high'] - df_ticker['close'].shift()).abs()
        low_close = (df_ticker['low'] - df_ticker['close'].shift()).abs()
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df_ticker['atr'] = tr.rolling(window=window).mean()
        
        # Add Average Directional Index (ADX)
        up_move = df_ticker['high'] - df_ticker['high'].shift()
        down_move = df_ticker['low'].shift() - df_ticker['low']
        plus_dm = pd.Series(np.where((up_move > down_move) & (up_move > 0), up_move, 0), index=df_ticker.index)
        minus_dm = pd.Series(np.where((down_move > up_move) & (down_move > 0), down_move, 0), index=df_ticker.index)
        window_dx = 14
        df_ticker['plus_di'] = 100 * (plus_dm.rolling(window=window_dx).mean() / df_ticker['atr'])
        df_ticker['minus_di'] = 100 * (minus_dm.rolling(window=window_dx).mean() / df_ticker['atr'])
        df_ticker['dx'] = 100 * (abs(df_ticker['plus_di'] - df_ticker['minus_di']) / (df_ticker['plus_di'] + df_ticker['minus_di']))

        # Calculate ADX
        window = 14
        dx_rolling = df_ticker['dx'].rolling(window=window).mean()
        first_valid_index = dx_rolling.first_valid_index()
        df_ticker['adx'] = np.nan
        df_ticker.loc[first_valid_index, 'adx'] = dx_rolling[first_valid_index]
        first_valid_index = df_ticker.index.get_loc(first_valid_index)
        for i in range(first_valid_index + 1, len(df_ticker)):
            df_ticker.loc[df_ticker.index[i], 'adx'] = (
                (df_ticker.loc[df_ticker.index[i - 1], 'adx'] * (window - 1) + df_ticker.loc[df_ticker.index[i], 'dx']) / window
            )

        # Add Bollinger Bands
        window = 20
        df_ticker['bb_middle'] = df_ticker['close'].rolling(window=window).mean()
        df_ticker['bb_std'] = df_ticker['close'].rolling(window=window).std()
        df_ticker['bb_upper'] = df_ticker['bb_middle'] + (df_ticker['bb_std'] * 2)
        df_ticker['bb_lower'] = df_ticker['bb_middle'] - (df_ticker['bb_std'] * 2)
        df_ticker['bb_pct'] = (df_ticker['close'] - df_ticker['bb_lower']) / (df_ticker['bb_upper'] - df_ticker['bb_lower'])
        df_ticker['bb_width'] = (df_ticker['bb_upper'] - df_ticker['bb_lower']) / df_ticker['bb_middle']
        df_ticker['bb_d_to_upper'] = df_ticker['close'] - df_ticker['bb_upper']
        df_ticker['bb_d_to_lower'] = df_ticker['close'] - df_ticker['bb_lower']

        # Add On-Balance Volume (OBV)
        df_ticker['obv'] = 0
        for i in range(1, len(df_ticker)):
            if df_ticker['close'].iloc[i] > df_ticker['close'].iloc[i-1]:
                df_ticker.loc[df_ticker.index[i], 'obv'] = df_ticker['obv'].iloc[i-1] + df_ticker['volume'].iloc[i]
            elif df_ticker['close'].iloc[i] < df_ticker['close'].iloc[i-1]:
                df_ticker.loc[df_ticker.index[i], 'obv'] = df_ticker['obv'].iloc[i-1] - df_ticker['volume'].iloc[i]
            else:
                df_ticker.loc[df_ticker.index[i], 'obv'] = df_ticker['obv'].iloc[i-1]
        df_ticker['obv_norm'] = (df_ticker['obv'] - df_ticker['obv'].min()) / (df_ticker['obv'].max() - df_ticker['obv'].min())
        df_ticker['obv_momentum'] = df_ticker['obv'] - df_ticker['obv'].shift(10)

        # Add Volume Price Trend (VPT)
        df_ticker['vpt'] = 0
        vpt_change = df_ticker['close'].pct_change(fill_method=None) * df_ticker['volume']
        df_ticker['vpt'] = df_ticker['vpt'].shift() + vpt_change
        df_ticker['vpt_norm'] = (df_ticker['vpt'] - df_ticker['vpt'].min()) / (df_ticker['vpt'].max() - df_ticker['vpt'].min())
        df_ticker['vpt_momentum'] = df_ticker['vpt'] - df_ticker['vpt'].shift(10)





        # ---- Add macro-economic indicators ---- #

        # Add S&P 500 Index
        df_ticker['sp500'] = df[df['symbol'] == 'SPY']['close'].copy()

        # Add calendaric features
        df_ticker.reset_index(inplace=True)
        df_ticker.set_index('timestamp', inplace=True)
        df_ticker.index = pd.to_datetime(df_ticker.index)
        df_ticker.index = df_ticker.index.date
        df_ticker.index = pd.to_datetime(df_ticker.index)
        df_ticker['dayofweek'] = df_ticker.index.day_of_week
        df_ticker['dayofyear'] = df_ticker.index.day_of_year
        df_ticker['month'] = df_ticker.index.month
        df_ticker['year'] = df_ticker.index.year
        df_ticker['quarter'] = df_ticker.index.quarter

        # Add macro-economic indicators that we prepared earlier in the pipeline
        df_ticker = df_ticker.join(df_macro, how='left')

        # Add the target variable which is the next day's close price
        df_ticker['target'] = df_ticker['close'].shift(-1)

        # we added return and log return to test out different approaches but this is not used in the thesis.
        df_ticker['target_ret'] = (df_ticker['target'] / df_ticker['close']) * 100
        df_ticker['target_ret_log'] = np.log(df_ticker['target'] / df_ticker['close'])
        
        df_bin.append(df_ticker.copy())

    chosen_stocks.append(df_bin.copy())

In [17]:
# check for missing values due to a lot of shifting during feature engineering
for column in chosen_stocks[1][14].columns:
    print(f"{column}: {chosen_stocks[1][14][column].isna().sum()}")

close: 0
high: 0
low: 0
trade_count: 0
open: 0
volume: 0
vwap: 0
symbol: 0
rsi: 14
roc_9: 9
roc_14: 14
roc_21: 21
macd: 0
macd_signal: 0
macd_hist: 0
stoch_%K: 13
stoch_%D: 15
stoch_diff: 15
cci: 28
ma_15: 4
ma_30: 9
ma_60: 14
ma_100: 19
ma_200: 29
atr: 13
plus_di: 13
minus_di: 13
dx: 13
adx: 26
bb_middle: 19
bb_std: 19
bb_upper: 19
bb_lower: 19
bb_pct: 19
bb_width: 19
bb_d_to_upper: 19
bb_d_to_lower: 19
obv: 0
obv_norm: 0
obv_momentum: 10
vpt: 1
vpt_norm: 1
vpt_momentum: 11
sp500: 0
dayofweek: 0
dayofyear: 0
month: 0
year: 0
quarter: 0
Unnamed: 0: 1256
real_GDP: 1256
unemployment_rate: 1256
CPI: 1256
personal_consumption: 1256
industrial_production: 1256
federal_funds_rate: 1256
10Y_treasury_rate: 1256
retail_sales: 1256
housing_starts: 1256
usd_eur: 1256
usd_gbp: 1256
usd_jpy: 1256
target: 1
target_ret: 1
target_ret_log: 1


In [18]:
# get a feeling for the data
chosen_stocks[1][13]

,close,high,low,trade_count,open,volume,vwap,symbol,rsi,roc_9,...,federal_funds_rate,10Y_treasury_rate,retail_sales,housing_starts,usd_eur,usd_gbp,usd_jpy,target,target_ret,target_ret_log
2020-05-08 00:00:00,5.6690,5.669,5.5640,49,5.575,16372,5.614000,EEA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5380,97.689187,-0.023379
2020-05-11 00:00:00,5.5380,5.640,5.5090,146,5.597,16710,5.577000,EEA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4290,98.031780,-0.019878
2020-05-12 00:00:00,5.4290,5.677,5.1960,243,5.487,281091,5.316000,EEA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.3200,97.992264,-0.020282
2020-05-13 00:00:00,5.3200,5.455,5.2980,232,5.429,57646,5.421000,EEA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2910,99.454887,-0.005466
2020-05-14 00:00:00,5.2910,5.298,5.2840,26,5.298,1844,5.292000,EEA,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2870,99.924400,-0.000756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01 00:00:00,9.5300,9.540,9.4451,42,9.540,5336,9.505202,EEA,66.777698,5.187638,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.6428,101.183631,0.011767
2025-05-02 00:00:00,9.6428,9.670,9.4713,55,9.550,8859,9.543689,EEA,69.597917,6.786268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.7500,101.111710,0.011056
2025-05-05 00:00:00,9.7500,9.750,9.6700,125,9.670,28301,9.709490,EEA,72.028133,6.093580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.6725,99.205128,-0.007980
2025-05-06 00:00:00,9.6725,9.870,9.5300,43,9.870,6546,9.699875,EEA,67.808110,4.624121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.6400,99.663996,-0.003366


In [19]:
# check all on lengths and missing values
for i in range(len(chosen_stocks)):
    print(f"Bin {i+1} contains {len(chosen_stocks[i])} stocks.")
    for j in range(len(chosen_stocks[i])):
        print(f"Date range: {chosen_stocks[i][j].index.min().date()} - {chosen_stocks[i][j].index.max().date()}")
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j])} rows.")
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j].columns)} columns.")
        print(f"Stock {j+1} in Bin {i+1} has missing values: {chosen_stocks[i][j].isnull().sum().sum()}")

Bin 1 contains 15 stocks.


AttributeError: 'float' object has no attribute 'date'

In [ ]:
copy = chosen_stocks.copy()

In [ ]:
# drop NaNs and check again 
for i in range(len(chosen_stocks)):
    print(f"Bin {i+1} contains {len(chosen_stocks[i])} stocks.")
    for j in range(len(chosen_stocks[i])):
        chosen_stocks[i][j].dropna(inplace=True)
        print(f"After dropping NaN values:")
        print(f"Date range: {chosen_stocks[i][j].index.min().date()} - {chosen_stocks[i][j].index.max().date()}")
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j])} rows.")
        print(f"Stock {j+1} in Bin {i+1} contains {len(chosen_stocks[i][j].columns)} columns.")
        print(f"Stock {j+1} in Bin {i+1} has missing values: {chosen_stocks[i][j].isnull().sum().sum()}")

Bin 1 contains 15 stocks.
After dropping NaN values:


AttributeError: 'float' object has no attribute 'date'

In [ ]:
chosen_stocks[2][14]
# that looks good

,close,high,low,trade_count,open,volume,vwap,symbol,rsi,roc_9,...,federal_funds_rate,10Y_treasury_rate,retail_sales,housing_starts,usd_eur,usd_gbp,usd_jpy,target,target_ret,target_ret_log
2020-05-08 00:00:00,127.11,128.950,126.3900,9679,128.95,572979,127.404147,LBRDK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.90,99.048069,-0.009565
2020-05-11 00:00:00,125.90,127.800,125.5100,10549,127.80,1904635,126.211521,LBRDK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.57,98.149325,-0.018680
2020-05-12 00:00:00,123.57,126.460,123.5700,6591,125.46,370238,124.753351,LBRDK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.92,98.664724,-0.013443
2020-05-13 00:00:00,121.92,124.150,119.7704,10607,124.15,692667,121.273991,LBRDK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.72,100.656168,0.006540
2020-05-14 00:00:00,122.72,122.930,120.2500,8866,120.69,454631,121.882798,LBRDK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.97,100.203716,0.002035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01 00:00:00,88.96,90.560,88.8000,16956,90.55,910700,89.316701,LBRDK,61.801403,13.730504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.55,100.663219,0.006610
2025-05-02 00:00:00,89.55,90.175,89.0800,14508,89.89,656744,89.425865,LBRDK,62.586252,19.879518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.78,102.490229,0.024597
2025-05-05 00:00:00,91.78,92.500,88.5900,16446,88.68,928323,91.251146,LBRDK,65.473774,19.551908,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.54,101.917629,0.018995
2025-05-06 00:00:00,93.54,93.650,90.4700,14619,90.47,795316,92.824823,LBRDK,67.599183,19.387364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.43,99.882403,-0.001177


In [ ]:
import pickle
# write out the preprocessed data
with open('preprocessed_data/preprocessed_general.pkl', 'wb') as f:
    pickle.dump(chosen_stocks, f)